In [15]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer_GraphCodeBert = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

model_GraphCodeBert = AutoModelForMaskedLM.from_pretrained("microsoft/graphcodebert-base")

In [3]:
import ast

def getLineAssignment(tree, lineno):
    return next((node for node in ast.walk(tree) if isinstance(node, ast.Name) and node.lineno == lineno), None)

def get_variables(expression):
    tree = ast.parse(expression)
    variables = []
    for node in ast.walk(tree):
        if not isinstance(node, ast.Assign):
            continue
        nodeVariable = getLineAssignment(tree, node.lineno).id
        if nodeVariable not in variables:
            variables.append(nodeVariable)
    return variables

In [4]:
class replaceName(ast.NodeTransformer):
    def __init__(self, varName, replaceName):
        self.varName = varName
        self.replaceName = replaceName

    def visit_Name(self, node):
        if(node.id == self.varName):
            node.id = self.replaceName
        return node


In [20]:
CODE = """
def Function(Variable):
    a = sin(Variable)
    b = cos(Variable)
    return a + b
"""

In [37]:
variables = get_variables(CODE)
masked = ast.unparse(replaceName(variables[1], '<mask>').visit(ast.parse(CODE)))
fill_mask_GCB = pipeline("fill-mask", model=model_GraphCodeBert, tokenizer=tokenizer_GraphCodeBert)
candidates = fill_mask_GCB(masked)
i = 0
while candidates[0][i]['token_str'][1:] == variables[1]:
    i += 1
replacement = candidates[0][i]['token_str'][1:]
new_code = ast.unparse(replaceName(variables[1], replacement).visit(ast.parse(CODE)))
print(CODE)
print(new_code)



def Function(Variable):
    a = sin(Variable)
    b = cos(Variable)
    return a + b

def Function(Variable):
    a = sin(Variable)
    c = cos(Variable)
    return a + c
